In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout, Output, Label
from IPython.display import display, Latex, Image, Markdown

import params as st
from model import *
from observer import *
from feedforward import *

import numpy as np
from scipy.signal import place_poles
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.integrate import solve_ivp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Regelung des Magnetlagers

In [ ]:
imag = Image("../images/magBall.png", width=125)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
**Modellgleichungen**
mit Störung $\\Delta i$ im Eingang
\\begin{align*}
    m \\ddot{y}(t) & = k \\frac{(i + \\Delta i)^2(t)}{(y(t)-s_0)^2} - m g
\\end{align*}

**flachheitsbasierter Regler**

Rückführung |   | Eingang
----------------------|----|------------------------
\\begin{align*}
    v(t) & = \\ddot{y}_{\\mathrm{r}}(t) - a_1 (\\dot{y}_{\\mathrm{r}}(t) - \\dot{y}(t)) - a_0 (y_{\\mathrm{r}}(t) - y(t)),\\\\
    & \\quad a_1,a_0 > 0
\\end{align*} |   | \\begin{align*}
    \\tilde{i}(t) & = (y_{\\mathrm{r}}(t) - s_0)\\sqrt{\\frac{m}{k} (v(t) + g)}
\\end{align*}


**Störgrößenkompensation mittels Beobachter**
\\begin{align*}
    \\dot{\\hat{\\mathbf{x}}}(t) & = \\begin{pmatrix}
    0 & 1 & 0 \\\\
    0 & 0 & 1 \\\\
    0 & 0 & 0
    \\end{pmatrix} \\hat{\\mathbf{x}}(t) + \\begin{pmatrix}
    0 \\\\  1 \\\\ 0
    \\end{pmatrix} v(t) + \\mathbf{l} (\\hat{y}(t) - y(t)), & \\hat{\\mathbf{x}}(t) & = \\begin{pmatrix}
    \\hat{y}(t) \\\\ \\dot{\\hat{y}}(t) \\\\ z(t)
    \\end{pmatrix}
\\end{align*}
mit Eingang
\\begin{align*}
    i(t) & = \\tilde{i}(t) - z(t)
\\end{align*}
    """))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 5, 501)

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(12, 6))
    ax3 = plt.subplot(222)
    ax4 = plt.subplot(224)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax11 = ax1.twinx()
ax1.set_xlim([tSim[0], tSim[-1]]) 
ax11.set_xlim([tSim[0], tSim[-1]]) 
ax2.set_xlim([tSim[0], tSim[-1]]) 
ax4.set_xlim([tSim[0], tSim[-1]]) 
ax1.grid() 
ax2.grid()
ax4.grid()
ax3.set_xlim(-2, 2)
ax3.set_ylim(-1, 3)
ax3.set_xticks([])
ax3.set_yticks([])
ax1.set_ylabel(r"$u$ in A")
ax11.set_ylabel(r"$\Delta i$", color='C2')
ax2.set_xlabel(r"$t$ in s")
ax2.set_ylabel(r"$y$ in m")
ax4.set_ylabel(r"$\dot{y}$ in m/s")
ax4.set_xlabel(r"$t$ in s")

lineY, = ax2.plot([], [], label=r"Istverlauf")
lineYref, = ax2.plot([], [], '--', label=r"Sollverlauf")
lineYobs, = ax2.plot([], [], '-.', label=r"beobachteter Verlauf")
linedY, = ax4.plot([], [])
linedYref, = ax4.plot([], [], '--')
linedYobs, = ax4.plot([], [], '-.')
lineU, = ax1.plot([], [])
lineUref, = ax1.plot([], [], '--')
linedI, = ax11.plot([], [], 'C2-.')

ballAni = ax3.add_patch(Ellipse((0, 0), st.r, st.b, facecolor='0.5', edgecolor='0.25'))
magAni = ax3.add_patch(plt.Rectangle((-st.r, st.r + st.s), 2 * st.r, st.h, facecolor='0.', edgecolor='0.'))

handlesAx, labelsAx = ax2.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=3, mode="expand", borderaxespad=0., framealpha=0.5)

playB = widgets.Play(value=0,
                     min=0, 
                     max=len(tSim),
                     step=10)
sliderB = widgets.IntSlider(value=0,
                            min=0,
                            max=len(tSim),
                            step=10)
sliderT0 = widgets.FloatSlider(value=0,
                               min=0,
                               max=5,
                               step=1,
                               description=r'$t_0$')
sliderT = widgets.FloatSlider(value=1,
                               min=0.5,
                               max=4,
                               step=.5,
                               description=r'$T$')
sliderYd = widgets.widgets.FloatSlider(value=0.5,
                                       min=0.2,
                                       max=0.7,
                                       step=0.1,
                                       description=r"$y_\text{d}$")
sliderY0 = widgets.widgets.FloatSlider(value=1,
                                       min=0.5,
                                       max=1.1,
                                       step=0.1,
                                       description=r"$y_\text{0}$")
sliderW0 = widgets.widgets.FloatSlider(value=0.2,
                                       min=0,
                                       max=2,
                                       step=0.05,
                                       description=r"$\omega_0$")
sliderD = widgets.widgets.FloatSlider(value=2,
                                      min=0,
                                      max=8.0,
                                      step=0.1,
                                      description=r"$D$")
sliderObs = widgets.Checkbox(value=False,
                             description='Aktiv',
                             disabled=False)
sliderObsY0 = widgets.widgets.FloatSlider(value=1,
                                          min=0.5,
                                          max=1.1,
                                          step=0.1,
                                          description=r"$y_\text{0}$")
sliderSimY0 = widgets.widgets.FloatSlider(value=1,
                                          min=0.5,
                                          max=1.1,
                                          step=0.1,
                                          description=r"$y_\text{0}$")
sliderSimDi = widgets.widgets.FloatSlider(value=0,
                                          min=-0.5,
                                          max=0.5,
                                          step=0.1,
                                          description=r"$\Delta i$")
def updateOde(_):
    global res

    t0Ff = sliderT0.value
    TFf = sliderT.value
    ydFf = sliderYd.value
    y0Ff = sliderY0.value
    y0Sim = sliderSimY0.value
    diSim = sliderSimDi.value
    y0Obs = sliderObsY0.value
    obs = sliderObs.value
    w0 = sliderW0.value
    D = sliderD.value

    def sys(t, x, yr, di, D, w0, obs, l, params):
        if obs:
            y = x[2]
            dy = x[3]
        else:
            y = x[0]
            dy = x[1]

        # Steuerung & Regler
        ddy = lambda t: yr[2](t) - 2 * D * w0 * (dy - yr[1](t)) - w0 ** 2 * (y - yr[0](t))
        # Linearisierende Zustandsrückführung
        if obs:
            u = lambda t: (y - st.s0) * np.sqrt(st.m / st.k * (ddy(t) + st.g)) + di - x[4]
            uObs = lambda t: (y - st.s0) * np.sqrt(st.m / st.k * (ddy(t) + st.g)) - x[4]
        else:
            u = lambda t: (y - st.s0) * np.sqrt(st.m / st.k * (ddy(t) + st.g)) + di
            uObs = lambda t: (y - st.s0) * np.sqrt(st.m / st.k * (ddy(t) + st.g))

        dx = np.zeros(5)
        # Model
        dx[0:2] = nonlinSys(t, x[0:2], u, params)
        # Beobachter
        dx[2:5] = nonlinObs(t, x[2:5], uObs, l, x[0], params)
        
        return dx

    params = st.g, st.m, st.s0 , st.k
    x0 = [y0Sim, 0, y0Obs, 0, 0]
    yr, _ = feedForwardFlat(t0Ff, TFf, ydFf, y0Ff)
    # Beobachterverstärkung
    l = place_poles(np.array([[0, 1, 0],[0, 0, 1],[0, 0, 0]]).T, np.array([[1, 0, 0]]).T, np.array([-2.5, -8, -20])).gain_matrix[0]

    res = solve_ivp(sys,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim,
                    args=(yr, diSim, w0, D, obs, l, params))

def updatePlot(change):
    idx = change['new']

    y = res.y.T[idx, 0]
    dy = res.y.T[idx, 1]
    
    t0Ff = sliderT0.value
    TFf = sliderT.value
    ydFf = sliderYd.value
    y0Ff = sliderY0.value
    y0Sim = sliderSimY0.value
    diSim = sliderSimDi.value
    y0Obs = sliderObsY0.value
    obs = sliderObs.value
    w0 = sliderW0.value
    D = sliderD.value

    yr, uFF = feedForwardFlat(t0Ff, TFf, ydFf, y0Ff)

    if obs:
        ddy = lambda t, idx: yr[2](t) - 2 * D * w0 * (res.y.T[idx, 3] - yr[1](t)) - w0 ** 2 * (res.y.T[idx, 2] - yr[0](t))
        u = lambda t, idx: (res.y.T[idx, 2] - st.s0) * np.sqrt(st.m / st.k * (ddy(t, idx) + st.g)) + diSim + res.y.T[idx, 4]
    else:
        ddy = lambda t, idx: yr[2](t) - 2 * D * w0 * (res.y.T[idx, 1] - yr[1](t)) - w0 ** 2 * (res.y.T[idx, 0] - yr[0](t))
        u = lambda t, idx: (res.y.T[idx, 0] - st.s0) * np.sqrt(st.m / st.k * (ddy(t, idx) + st.g)) + diSim

    lineU.set_data(tSim[:idx], [u(_t, _i) for _i, _t in enumerate(tSim[:idx])])
    lineUref.set_data(tSim[:idx], [uFF(_t) for _t in tSim[:idx]])
    linedI.set_data(tSim[:idx], res.y.T[:idx, 4])
    lineY.set_data(tSim[:idx], res.y.T[:idx, 0])
    lineYref.set_data(tSim[:idx], [yr[0](_t) for _t in tSim[:idx]])
    lineYobs.set_data(tSim[:idx], res.y.T[:idx, 2])
    linedY.set_data(tSim[:idx], res.y.T[:idx, 1])
    linedYref.set_data(tSim[:idx], [yr[1](_t) for _t in tSim[:idx]])
    linedYobs.set_data(tSim[:idx], res.y.T[:idx, 3])

    iMin = np.min(res.y.T[:, 4])
    iMax = np.max(res.y.T[:, 4])
    uMin = np.minimum(np.min([uFF(_t) for _t in tSim]), np.min([u(_t, _i) for _i, _t in enumerate(tSim)]))
    uMax = np.maximum(np.max([uFF(_t) for _t in tSim]), np.max([u(_t, _i) for _i, _t in enumerate(tSim)]))
    yMin = np.minimum(np.minimum(np.min(res.y.T[:, 0]), np.min(res.y.T[:, 2])), np.min([yr[0](_t) for _t in tSim]))
    yMax = np.maximum(np.maximum(np.max(res.y.T[:, 0]), np.max(res.y.T[:, 2])), np.max([yr[0](_t) for _t in tSim]))
    dyMin = np.minimum(np.minimum(np.min(res.y.T[:, 1]), np.min(res.y.T[:, 3])), np.min([yr[1](_t) for _t in tSim]))
    dyMax = np.maximum(np.maximum(np.max(res.y.T[:, 1]), np.max(res.y.T[:, 2])), np.max([yr[1](_t) for _t in tSim]))
    ax1.set_ylim(uMin - np.abs(uMax - uMin) * 0.1, uMax + np.abs(uMax - uMin) * 0.1)
    ax11.set_ylim(iMin - np.abs(iMax - iMin) * 0.1, iMax + np.abs(iMax - iMin) * 0.1)
    ax2.set_ylim(yMin - np.abs(yMax - yMin) * 0.1, yMax + np.abs(yMax - yMin) * 0.1)
    ax4.set_ylim(dyMin - np.abs(dyMax - dyMin) * 0.1, dyMax + np.abs(dyMax - dyMin) * 0.1)

    ballAni.set_center((0, y - 1))
    
    fig.canvas.draw()    

sliderB.observe(updatePlot, names='value')
sliderT0.observe(updateOde, names='value')
sliderT.observe(updateOde, names='value')
sliderYd.observe(updateOde, names='value')
sliderY0.observe(updateOde, names='value')
sliderSimY0.observe(updateOde, names='value')
sliderSimDi.observe(updateOde, names='value')
sliderObsY0.observe(updateOde, names='value')
sliderObs.observe(updateOde, names='value')
sliderW0.observe(updateOde, names='value')
sliderD.observe(updateOde, names='value')

updateOde(_)

bFeedForward = VBox([Label(value='Steuerung'), sliderY0, sliderYd, sliderT0, sliderT])
bFeedForward.layout = make_box_layout()
bFeedBack = VBox([Label(value='Regler'), sliderW0, sliderD])
bFeedBack.layout = make_box_layout()
bObserver = VBox([Label(value='Beobachter'), sliderObs, sliderObsY0])
bObserver.layout = make_box_layout()
bModel = VBox([Label(value='Modell'), sliderSimY0, sliderSimDi])
bModel.layout = make_box_layout()
controls = HBox([bFeedForward, bFeedBack, bObserver, bModel])

jslink((playB, 'value'), (sliderB, 'value'))
videoControls = VBox([HBox([playB, sliderB]), output])
videoControls.layout = make_box_layout()

HBox([controls, Box([videoControls])], layout=Layout(display='flex', flex_flow='column', justify_content='center', align_items='center'))